In [2]:
pip install tqdm fitz PyMuPDF

  Using cached nipype-1.8.6-py3-none-any.whl.metadata (6.6 kB)
     ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
     ---------------------------------------- 0.2/42.2 MB 4.6 MB/s eta 0:00:10
     ---------------------------------------- 0.5/42.2 MB 5.2 MB/s eta 0:00:08
      --------------------------------------- 0.8/42.2 MB 6.5 MB/s eta 0:00:07
     - -------------------------------------- 1.5/42.2 MB 8.1 MB/s eta 0:00:06
     -- ------------------------------------- 2.2/42.2 MB 8.7 MB/s eta 0:00:05
     -- ------------------------------------- 2.7/42.2 MB 9.0 MB/s eta 0:00:05
     --- ------------------------------------ 3.2/42.2 MB 9.4 MB/s eta 0:00:05
     --- ------------------------------------ 3.7/42.2 MB 9.5 MB/s eta 0:00:05
     ---- ----------------------------------- 4.3/42.2 MB 9.7 MB/s eta 0:00:04
     ---- ----------------------------------- 4.8/42.2 MB 9.9 MB/s eta 0:00:04
     ----- ---------------------------------- 5.3/42.2 MB 10.0 MB/s eta 0:0

In [7]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [16]:
!nvidia-smi

Sat Feb 17 10:56:25 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   34C    P8             12W /  165W |     599MiB /  16380MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
import torch
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm



device = "cuda"

# Definir una clase para tu conjunto de datos personalizado
class MyDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=128):
        self.data = []
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Leer tus datos y procesarlos
        with open(data_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()
                if line:
                    self.data.append(line)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Ruta a tus datos
data_path = "pdf/icp.txt"

# Tokenizador y modelo preentrenado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
#tokenizer.pad_token = tokenizer.eos_token
#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# Dataset y DataLoader
dataset = MyDataset(data_path, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Función de entrenamiento
def train_model(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    return total_loss / len(dataloader)

# Parámetros de entrenamiento
epochs = 3
learning_rate = 1e-5
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



In [20]:
# Entrenamiento
model.to(device)
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    avg_loss = train_model(model, dataloader, optimizer, device)
    print(f"Average Loss: {avg_loss:.4f}")


AssertionError: Torch not compiled with CUDA enabled

In [3]:
import torch
if torch.cuda.is_available():
    print("CUDA está disponible")
else:
    print("CUDA no está disponible")

CUDA no está disponible


In [4]:
CUDA_HOME

NameError: name 'CUDA_HOME' is not defined

In [2]:
pip install tensorflow

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
  Using cached cachetools-5.3.2-py3-none-any.whl.metadata (5.2 kB)
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     -------------------------------------- 151.7/151.7 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/276.5 MB ? eta -:--:--
   ---------------------------------------- 0.6/276.5 MB 12.2 MB/s eta 0:00:23
   ---------------------------------------- 1.1/276.5 MB 11.9 MB/s eta 0:00:24
   ---------------------------------------- 1.7/276.5 MB 11.9 MB/s eta 0:00:24
   ------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [4]:
import tensorflow as tf

# Verificar si hay una GPU disponible
if tf.config.list_physical_devices('GPU'):
    print('Se encontró una GPU')
else:
    print("No se encontró una GPU, el código se ejecutará en la CPU")

# Crear algunos datos de ejemplo
x = tf.random.uniform([1000, 1000])

# Realizar alguna operación en la GPU
with tf.device('/GPU:0'):
    result = tf.matmul(x, x)

print(result)


No se encontró una GPU, el código se ejecutará en la CPU
tf.Tensor(
[[266.6501  259.53195 256.20923 ... 254.09723 257.2446  253.53052]
 [253.48038 245.50024 245.00629 ... 243.14006 244.76425 241.18141]
 [262.9653  249.83011 251.77267 ... 250.23866 249.8279  246.95485]
 ...
 [266.409   248.94673 250.65134 ... 248.91614 256.27292 247.80475]
 [249.02214 246.74458 245.8597  ... 241.70242 243.21176 241.52928]
 [254.33243 255.79095 248.74556 ... 245.15306 244.43204 242.49219]], shape=(1000, 1000), dtype=float32)
